In [1]:
from main import generate_estimation
from math import ceil, floor
import time
import json
from datetime import datetime
import threading
import os
import numpy as np
# _, numOfCars = generate_estimation()
# print(_)
# numOfCars
# greenTime = min(numOfCars * 2 + 3, 120)
# print(numOfCars, greenTime)

In [2]:
# write code to remove the content of intersection_A.json, intersection_B.json, intersection_C.json, intersection_D.json
# after the simulation is done
#check if file is existed
if os.path.exists("intersection_A.json"):
    with open("intersection_A.json", "w") as file:
        file.write("")
if os.path.exists("intersection_B.json"):
    with open("intersection_B.json", "w") as file:
        file.write("")
if os.path.exists("intersection_C.json"):
    with open("intersection_C.json", "w") as file:
        file.write("")
if os.path.exists("intersection_D.json"):
    with open("intersection_D.json", "w") as file:
        file.write("")
if os.path.exists("merged_sorted_intersections.json"):
    with open("merged_sorted_intersections.json", "w") as file:
        file.write("")


In [3]:
class TrafficLight:
    def __init__(self, trafficID, lanes):
        self.trafficID = trafficID
        self.lanes = lanes
        self.totalCars = 0
        self.carsPerLane = 0
        self.greenTime = 3

    def update_traffic(self, totalCars):
        self.totalCars += totalCars
        self.carsPerLane = ceil(totalCars / self.lanes)
        # self.greenTime = min(self.carsPerLane * 2 + 3, 50)
        self.greenTime = max(min(ceil((self.totalCars * 2.5)/(self.lanes +1)), 50),3)
    def getTotalCars(self):
        return self.totalCars
    def getGreenTime(self):
        return self.greenTime
    def getTrafficID(self):
        return self.trafficID
    def resetGreenTime(self):
        self.greenTime = 3
    def resetTotalCars(self):
        if self.greenTime > 3:
            self.totalCars -= floor(max(0, (self.greenTime*(self.lanes + 1))//2.5))
            self.totalCars = max(0, self.totalCars)
        else:
            self.totalCars = 0
    def __str__(self):
        return (f"Traffic Light {self.trafficID} -> Lanes: {self.lanes}, Total Cars: {self.totalCars}, "
                f"Cars per Lane: {self.carsPerLane}, Green Time: {self.greenTime}s")
       # totalCars = 20  # Simulated number of cars

class Intersection:
    def __init__(self, intersectionID, trafficLights):
        self.intersectionID = intersectionID
        self.trafficLights = trafficLights
    def getIntersectionID(self):
        return self.intersectionID
    def getTrafficLights(self):
        return self.trafficLights
    def getSignalOn(self):
        for light in self.trafficLights:
            if light.greenTime > 0:
                return light.getTrafficID
        return None
    def getTrafficLightwithID(self, trafficID):
        for light in self.trafficLights:
            if light.getTrafficID() == trafficID:
                return light
        return None


class CircularTrafficLights:
    def __init__(self, intersection):
        self.lights = intersection.getTrafficLights()
        self.intersection = intersection
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if not self.lights:
            raise StopIteration
        light = self.lights[self.index]
        self.index = (self.index + 1) % len(self.lights)
        return light
    def getIntersection(self):
        return self.intersection


    # def update_light(self, trafficID, totalCars, carsPerLane, greenTime):
    #     for light in self.lights:
    #         if light.trafficID == trafficID:
    #             light.update_traffic(totalCars, carsPerLane, greenTime)
    #             print(f"Updated Traffic Light {trafficID}")

def save_to_database(data, filename='testA.json'):
    try:
        with open(filename, 'r') as json_file:
            existing_data = json.load(json_file)
            if not isinstance(existing_data, list):
                existing_data = []
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    
    data_dict = {
        f"Cycle {data[0]}": {
            "trafficID": data[1],
            "num_cars": data[2],
            "weatherStamp:": data[3],
            "timeStamp": data[4],
            "dateStamp": data[5],
            "greenTime": data[6]
        }
    }

    existing_data.append(data_dict)  # Use append to add the dictionary to the list

    with open(filename, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

In [4]:


intersection1 = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]
intersection2 = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]
intersection3 = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]
intersection4 = [
    TrafficLight(1, 4), # trafficID, lanes
    TrafficLight(2, 4),
    TrafficLight(3, 4),
    TrafficLight(4, 4)
]
network = [Intersection('A',intersection1), Intersection('B',intersection2), Intersection('C',intersection3), Intersection('D',intersection4)]
# traffic_circle = []
# for i, intersection in enumerate(network):
#     traffic_circle.append(CircularTrafficLights(intersection))

traffic_circle1 = CircularTrafficLights(network[0])
traffic_circle2 = CircularTrafficLights(network[1])
traffic_circle3 = CircularTrafficLights(network[2])
traffic_circle4 = CircularTrafficLights(network[3])

In [5]:
def setup_delayed_task(func, delay, *args):
    timer = threading.Timer(delay, func, args=args)
    timer.start()
    return
def procesIntersectionWithThermal(circularTrafficLights):
    i = 0
    cycle = 0
    delay_seconds = 30 
    for light in circularTrafficLights:
        if(i % len(circularTrafficLights.getIntersection().getTrafficLights()) == 0):
            cycle += 1
        if(cycle == 1000):
            break
        i += 1
        time.sleep(2)
        while True:
            try:
                history, numOfCars = generate_estimation()
                numOfCars = np.random.randint(0, 120)  # Simulate number of cars
                break  # Exit the loop if the function succeeds
            except Exception as e:
                print(f"Error occurred: {e}. Retrying...")

        light.update_traffic(numOfCars)
        weatherStamp = history[0][2]
        timeStamp = datetime.now().strftime("%H:%M:%S")
        dateStamp = datetime.now().strftime("%Y-%m-%d")

        data = [
            cycle, light.trafficID, numOfCars,
            weatherStamp, timeStamp, dateStamp,
            light.getGreenTime()
        ]
        save_to_database(data, f"intersection_{circularTrafficLights.getIntersection().getIntersectionID()}.json")
        print(light)
        time.sleep(light.getGreenTime())
        if circularTrafficLights.getIntersection().getIntersectionID() == "A":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "B":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "C":
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
        elif circularTrafficLights.getIntersection().getIntersectionID() == "D":
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(), light.getTrafficID(), light.getTotalCars())
        light.resetTotalCars()
        light.resetGreenTime()


def processIntersectionWithSensors(circularTrafficLights):

    i = 0
    cycle = 0
    delay_seconds = 30
    for light in circularTrafficLights:
        if(i % len(circularTrafficLights.getIntersection().getTrafficLights()) == 0):
            cycle += 1
        if(cycle == 1000):
            break
        i += 1
        time.sleep(2)
        
        weatherStamp = None
        timeStamp = datetime.now().strftime("%H:%M:%S")
        dateStamp = datetime.now().strftime("%Y-%m-%d")
        numOfCars = light.getTotalCars()
        data = [
            cycle, light.trafficID, numOfCars,
            weatherStamp, timeStamp, dateStamp,
            light.getGreenTime()
        ]
        save_to_database(data, f"intersection_{circularTrafficLights.getIntersection().getIntersectionID()}.json")
        time.sleep(light.getGreenTime())
        if circularTrafficLights.getIntersection().getIntersectionID() == "A":
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getGreenTime(), light.lanes)
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getGreenTime(), light.lanes)
        elif circularTrafficLights.getIntersection().getIntersectionID() == "B":
            setup_delayed_task(L1, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getGreenTime(), light.lanes)
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getGreenTime(), light.lanes)
        elif circularTrafficLights.getIntersection().getIntersectionID() == "C":
            setup_delayed_task(L2, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getGreenTime(), light.lanes)
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getGreenTime(), light.lanes)
        elif circularTrafficLights.getIntersection().getIntersectionID() == "D":
            setup_delayed_task(L3, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getGreenTime(), light.lanes)
            setup_delayed_task(L4, delay_seconds, circularTrafficLights.getIntersection(),light.getTrafficID(), light.getGreenTime(), light.lanes)
        light.resetTotalCars()
        light.resetGreenTime()
def L1(intersection,trafficLightID, greenTime, lanes):
    totalCars =floor(max(0, (greenTime*(lanes + 1))//2.5)) 
    if(intersection.getIntersectionID() == "A"):
        if(trafficLightID == 1):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 2):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[1].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "B"):
        if(trafficLightID == 1):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[0].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))

def L2(intersection,trafficLightID, greenTime, lanes):
    totalCars = floor(max(0, (greenTime*(lanes + 1))//2.5))
    if(intersection.getIntersectionID() == "A"):
        if(trafficLightID == 1):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 2):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[2].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "C"):
        if(trafficLightID == 2):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 3):
            network[0].getTrafficLightwithID(3).update_traffic(int(0.8 * totalCars))

def L3(intersection,trafficLightID, greenTime, lanes):
    totalCars = floor(max(0, (greenTime*(lanes + 1))//2.5))
    if(intersection.getIntersectionID() == "C"):
        if(trafficLightID == 1):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 2):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[3].getTrafficLightwithID(2).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "D"):
        if(trafficLightID == 1):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 4):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 3):
            network[2].getTrafficLightwithID(4).update_traffic(int(0.2 * totalCars))

def L4(intersection,trafficLightID, greenTime, lanes):
    totalCars = floor(max(0, (greenTime*(lanes + 1))//2.5))
    if(intersection.getIntersectionID() == "B"):
        if(trafficLightID == 2):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 1):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 4):
            network[3].getTrafficLightwithID(1).update_traffic(int(0.2 * totalCars))
    elif(intersection.getIntersectionID() == "D"):
        if(trafficLightID == 2):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))
        elif(trafficLightID == 3):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.8 * totalCars))
        elif(trafficLightID == 4):
            network[1].getTrafficLightwithID(3).update_traffic(int(0.2 * totalCars))

In [6]:
threadA = threading.Thread(target=procesIntersectionWithThermal, args=(traffic_circle1,))
threadB = threading.Thread(target=processIntersectionWithSensors, args=(traffic_circle2,))
threadC = threading.Thread(target=processIntersectionWithSensors, args=(traffic_circle3,))
threadD = threading.Thread(target=procesIntersectionWithThermal, args=(traffic_circle4,))

threadA.start()
threadB.start()
threadC.start()
threadD.start()

threadA.join()
threadB.join()
threadC.join()
threadD.join()


Traffic Light 1 -> Lanes: 4, Total Cars: 50, Cars per Lane: 13, Green Time: 25s
Traffic Light 1 -> Lanes: 4, Total Cars: 38, Cars per Lane: 10, Green Time: 19s
Traffic Light 2 -> Lanes: 4, Total Cars: 47, Cars per Lane: 12, Green Time: 24s
Traffic Light 2 -> Lanes: 4, Total Cars: 85, Cars per Lane: 22, Green Time: 43s


Exception in thread Exception in thread Thread-25:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
Thread-26:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
TypeError: L4() missing 1 required positional argument: 'lanes'
    self.function(*self.args, **self.kwargs)
TypeError: L3() missing 1 required positional argument: 'lanes'


Traffic Light 3 -> Lanes: 4, Total Cars: 64, Cars per Lane: 16, Green Time: 32s


Exception in thread Exception in thread Thread-32:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
Thread-31:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.function(*self.args, **self.kwargs)
TypeError: L2() missing 1 required positional argument: 'lanes'
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
TypeError: L1() missing 1 required positional argument: 'lanes'


Traffic Light 3 -> Lanes: 4, Total Cars: 123, Cars per Lane: 28, Green Time: 50s


...

Exception in thread Thread-8937:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-8938:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
TypeError: L4() missing 1 required positional argument: 'lanes'
    self.function(*self.args, **self.kwargs)
TypeError: L3() missing 1 required positional argument: 'lanes'


Traffic Light 4 -> Lanes: 4, Total Cars: 51, Cars per Lane: 13, Green Time: 26s


Exception in thread Exception in thread Thread-8959:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
Thread-8960:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
TypeError: L1() missing 1 required positional argument: 'lanes'
    self.function(*self.args, **self.kwargs)
TypeError: L2() missing 1 required positional argument: 'lanes'
Exception in thread Thread-8966:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
Exception

Traffic Light 1 -> Lanes: 4, Total Cars: 854, Cars per Lane: 15, Green Time: 50s
Traffic Light 4 -> Lanes: 4, Total Cars: 505, Cars per Lane: 2, Green Time: 50s


Exception in thread Exception in thread Thread-8988:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
Thread-8987:
Traceback (most recent call last):
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.run()
  File "c:\Users\omars\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
TypeError: L4() missing 1 required positional argument: 'lanes'
    self.function(*self.args, **self.kwargs)
TypeError: L3() missing 1 required positional argument: 'lanes'


In [ ]:
# cnt = 0
# for light in traffic_circle1:
#     print(light.)
#     cnt+=1
#     if cnt == 4:
#         break

In [ ]:
def update_traffic_ids(file_path, intersection_id):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for cycle in data:
        for cycle_name, cycle_data in cycle.items():
            traffic_id = cycle_data['trafficID']
            cycle_data['trafficID'] = f"{intersection_id}{traffic_id}"
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

# File path for intersection_A.json
for intersection in network:
    file_path = f'intersection_{intersection.getIntersectionID()}.json'
    update_traffic_ids(file_path, intersection.getIntersectionID())


print("Traffic IDs updated successfully.")


Traffic IDs updated successfully.


In [ ]:
import glob

# Function to load and merge JSON files
def load_and_merge_json(file_paths):
    merged_data = []
    for file_path in file_paths:
        with open(file_path, 'r') as f:
            data = json.load(f)
            for entry in data:
                for cycle_data in entry.values():
                    merged_data.append(cycle_data)
    return merged_data

# Function to sort data based on dateStamp and timeStamp
def sort_data(data):
    sorted_data = sorted(data, key=lambda x: (x['dateStamp'], x['timeStamp']))
    return sorted_data

# List of JSON files
file_paths = ['intersection_A.json', 'intersection_B.json', 'intersection_C.json', 'intersection_D.json']

# Load and merge data
merged_data = load_and_merge_json(file_paths)

# Sort the merged data
sorted_data = sort_data(merged_data)

# Save the sorted data to a new JSON file
output_file = 'merged_sorted_intersections.json'
with open(output_file, 'w') as f:
    json.dump(sorted_data, f, indent=4)

print(f"Merged and sorted data saved to {output_file}")


Merged and sorted data saved to merged_sorted_intersections.json


In [ ]:
import json
import csv

# Load the merged and sorted JSON data
input_file = 'merged_sorted_intersections.json'
with open(input_file, 'r') as f:
    data = json.load(f)

# Define the output CSV file
output_file = 'merged_sorted_intersections.csv'

# Define the CSV columns
columns = ['trafficID', 'num_cars', 'weatherStamp:', 'timeStamp', 'dateStamp', 'greenTime']

# Write data to CSV
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()
    for entry in data:
        writer.writerow(entry)

print(f"Data has been written to {output_file}")


Data has been written to merged_sorted_intersections.csv


In [ ]:
import pandas as pd
#oopen csv file and display first 10 raws
df = pd.read_json('merged_sorted_intersections.json')
print(df.head(10))


  trafficID  num_cars  weatherStamp:           timeStamp   dateStamp  \
0        A1        92           30.0 2024-07-30 09:44:39  2024-07-30   
1        B1         0            NaN 2024-07-30 09:44:39  2024-07-30   
2        C1         0            NaN 2024-07-30 09:44:39  2024-07-30   
3        D1        72           30.0 2024-07-30 09:44:39  2024-07-30   
4        B2         0            NaN 2024-07-30 09:44:44  2024-07-30   
5        C2         0            NaN 2024-07-30 09:44:44  2024-07-30   
6        B3         0            NaN 2024-07-30 09:44:49  2024-07-30   
7        C3         0            NaN 2024-07-30 09:44:49  2024-07-30   
8        B4         0            NaN 2024-07-30 09:44:54  2024-07-30   
9        C4         0            NaN 2024-07-30 09:44:54  2024-07-30   

   greenTime  
0         46  
1          3  
2          3  
3         36  
4          3  
5          3  
6          3  
7          3  
8          3  
9          3  


In [ ]:
len(df)

1430